<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_02/TS%20-%20W2%20-%2002%20-%20Prepara%C3%A7%C3%A3o%20de%20recursos%20e%20r%C3%B3tulos%20(Laborat%C3%B3rio%201).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Preparação de recursos e rótulos de séries temporais

Neste laboratório, você preparará dados de séries temporais em recursos e rótulos que poderão ser usados para treinar um modelo.

Isso é feito principalmente por meio de uma técnica de *janelamento*(_windowing_), na qual você agrupa valores de medição consecutivos em um recurso e a próxima medição será o rótulo.

Por exemplo, em medições horárias, você pode usar os valores obtidos nas horas 1 a 11 para prever o valor na hora 12.

As próximas seções mostrarão como você pode implementar isso no Tensorflow.

Vamos começar!

## Importações

O Tensorflow será sua única importação neste módulo e você usará métodos principalmente da API [tf.data](https://www.tensorflow.org/guide/data), particularmente a classe [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).


Ela contém muitos métodos úteis para organizar sequências de dados e você verá isso em breve.

In [ ]:
import tensorflow as tf

## Criar um conjunto de dados simples

Para este exercício, você usará apenas uma sequência de números como conjunto de dados para que possa ver claramente o efeito de cada comando.

Por exemplo, a célula abaixo usa o método [range()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#range) para gerar um conjunto de dados contendo números de 0 a 9.

In [ ]:
# Gerar um conjunto de dados tf com 10 elementos (ou seja, números de 0 a 9)
dataset = tf.data.Dataset.range(10)

# Visualizar o resultado
for val in dataset:
   print(val.numpy())

Você verá esse comando várias vezes nas próximas seções.

## Janelamento dos dados

Conforme mencionado anteriormente, você deseja agrupar elementos consecutivos de seus dados e usá-los para prever um valor futuro.

Isso é chamado de janelamento e você pode usá-lo com o método [window()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#window), conforme mostrado abaixo.
> Aqui, você utilizará 5 elementos por janela (ou seja, o parâmetro `size`) e moverá essa janela 1 elemento por vez (ou seja, o parâmetro `shift`).

Uma ressalva ao usar esse método é que cada janela retornada é um [Dataset](https://www.tensorflow.org/guide/data#dataset_structure) em si.
> Esse é um iterável Python e, a partir da versão TF 2.8, ele não mostrará os elementos se você usar o método `print()` nele. Ele mostrará apenas uma descrição da estrutura de dados (por exemplo, `<_VariantDataset shapes: (), types: tf.int64>`).

In [ ]:
# Gerar um conjunto de dados tf com 10 elementos (ou seja, números de 0 a 9)
dataset = tf.data.Dataset.range(10)

# Janela de dados
dataset = dataset.window(size=5, shift=1)

# Imprimir o resultado
for window_dataset in dataset:
  print(window_dataset)

Se quiser ver os elementos, você terá de iterar sobre cada iterável.
> Isso pode ser feito modificando a instrução print acima com um for-loop aninhado ou uma compreensão de lista(_list comprehension_).

O código abaixo mostra a compreensão de lista:

In [ ]:
# Imprimir o resultado
for window_dataset in dataset:
  print([item.numpy() for item in window_dataset])

Agora que você pode ver os elementos de cada janela, notará que os conjuntos resultantes não são dimensionados uniformemente porque não há mais elementos após o número `9`.

Você pode usar o sinalizador `drop_remainder` para garantir que apenas as janelas de 5 elementos sejam mantidas.

In [ ]:
# Gerar um conjunto de dados tf com 10 elementos (ou seja, números de 0 a 9)
dataset = tf.data.Dataset.range(10)

# Janela de dados, mas só pega aqueles com o tamanho especificado
dataset = dataset.window(size=5, shift=1, drop_remainder=True)

# Imprimir o resultado
for window_dataset in dataset:
  print([item.numpy() for item in window_dataset])

## Achatar as janelas

Ao treinar o modelo posteriormente, você desejará preparar as janelas para serem [tensores](https://www.tensorflow.org/guide/tensor) em vez da estrutura `Dataset`.
> Você pode fazer isso alimentando uma função de mapeamento com o método [flat_map()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#flat_map).

Essa função será aplicada a cada janela e os resultados serão [achatados em um único conjunto de dados](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#flatten_a_dataset_of_windows_2).

Para ilustrar, o código abaixo colocará todos os elementos de uma janela em um único lote e, em seguida, achatará o resultado.

In [ ]:
# Gerar um conjunto de dados tf com 10 elementos (ou seja, números de 0 a 9)
dataset = tf.data.Dataset.range(10)

# Janela de dados, mas só pega aqueles com o tamanho especificado
dataset = dataset.window(5, shift=1, drop_remainder=True)

# Achatar as janelas, colocando seus elementos em um único lote
dataset = dataset.flat_map(lambda window: window.batch(5))

# Imprimir os resultados
for window in dataset:
  print(window.numpy())

## Agrupar em recursos e rótulos

Em seguida, você deverá marcar os rótulos em cada janela.
> Para este exercício, você fará isso separando o último elemento de cada janela dos quatro primeiros.

Isso é feito com o método [map()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) que contém uma função lambda que define o fatiamento da janela.

In [ ]:
# Gerar um conjunto de dados tf com 10 elementos (ou seja, números de 0 a 9)
dataset = tf.data.Dataset.range(10)

# Janela de dados, mas só pega aqueles com o tamanho especificado
dataset = dataset.window(5, shift=1, drop_remainder=True)

# Achatar as janelas, colocando seus elementos em um único lote
dataset = dataset.flat_map(lambda window: window.batch(5))

# Crie tuplas com recursos (quatro primeiros elementos da janela) e rótulos (último elemento)
dataset = dataset.map(lambda window: (window[:-1], window[-1]))

# Imprimir os resultados
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())
  print()

## Embaralhe os dados

É uma boa prática embaralhar o conjunto de dados para reduzir o *enviesamento de sequência*(_sequence bias_) durante o treinamento do modelo.
> Isso se refere ao ajuste excessivo da rede neural à ordem das entradas e, consequentemente, ela não terá um bom desempenho quando não vir essa ordem específica durante o teste.

Você não quer que a sequência de entradas de treinamento afete a rede dessa forma, portanto, é bom embaralhá-las.

Você pode simplesmente usar o método [shuffle()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle) para fazer isso.
> O parâmetro `buffer_size` é necessário para isso e, conforme mencionado no documento, você deve colocar um número igual ou maior do que o número total de elementos para um melhor embaralhamento.

Podemos ver nas células anteriores que o número total de janelas no conjunto de dados é `6`, portanto, podemos escolher esse número ou um número maior.

In [ ]:
# Gerar um conjunto de dados tf com 10 elementos (ou seja, números de 0 a 9)
dataset = tf.data.Dataset.range(10)

# Janela de dados, mas só pega aqueles com o tamanho especificado
dataset = dataset.window(5, shift=1, drop_remainder=True)

# Achatar as janelas, colocando seus elementos em um único lote
dataset = dataset.flat_map(lambda window: window.batch(5))

# Crie tuplas com recursos (quatro primeiros elementos da janela) e rótulos (último elemento)
dataset = dataset.map(lambda window: (window[:-1], window[-1]))

# Embaralhar as janelas
dataset = dataset.shuffle(buffer_size=10)

# Imprimir os resultados
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())
  print()


## Criar lotes para treinamento

Por fim, você deverá agrupar suas janelas em lotes.

Você pode fazer isso com o método [batch()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch), conforme mostrado abaixo.

Basta especificar o tamanho do lote e ele retornará um conjunto de dados em lote com esse número de janelas.

Como regra geral, também é bom especificar uma etapa [prefetch()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch).
> Isso otimiza o tempo de execução quando o modelo já está sendo treinado.

Ao especificar um prefetch `buffer_size` de `1`, conforme mostrado abaixo, o Tensorflow preparará o próximo lote com antecedência (ou seja, colocando-o em um buffer) enquanto o lote atual estiver sendo consumido pelo modelo. Você pode ler mais sobre isso [aqui] (https://towardsdatascience.com/optimising-your-input-pipeline-performance-with-tf-data-part-1-32e52a30cac4#Prefetching).

In [ ]:
# Gerar um conjunto de dados tf com 10 elementos (ou seja, números de 0 a 9)
dataset = tf.data.Dataset.range(10)

# Janela de dados, mas só pega aqueles com o tamanho especificado
dataset = dataset.window(5, shift=1, drop_remainder=True)

# Achatar as janelas, colocando seus elementos em um único lote
dataset = dataset.flat_map(lambda window: window.batch(5))

# Crie tuplas com recursos (quatro primeiros elementos da janela) e rótulos (último elemento)
dataset = dataset.map(lambda window: (window[:-1], window[-1]))

# Embaralhar as janelas
dataset = dataset.shuffle(buffer_size=10)

# Embaralhar as janelas
dataset = dataset.batch(2).prefetch(1)

# Imprimir os resultados
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())
  print()


## Resumo

Este breve exercício mostrou a você como encadear diferentes métodos da classe `tf.data.Dataset` para preparar uma sequência em conjuntos de dados de janelas embaralhadas e em lote.

Você usará esse mesmo conceito nos próximos exercícios quando aplicá-lo a dados sintéticos e usar o resultado para treinar uma rede neural. Vamos para o próximo!